In [1]:
import os

import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer # SKT에서 GPT 만들때 이 토크나이저 사용해서..
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

아래 실행 커멘드는 gpt_ckpt 폴더가 있지 않은 경우에만 실행해주세요.

In [2]:
import wget
import zipfile

wget.download('https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/releases/download/v1.0/gpt_ckpt.zip')

with zipfile.ZipFile('gpt_ckpt.zip') as z:
    z.extractall()

In [3]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [4]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [5]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

In [6]:
tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

In [8]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])

        
    
    

In [15]:
def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],] + vocab[toked])[None, :]
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            # 확률이 가장 높은 단어만 선택
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            # 확률이 높은 순서대로 top-k번째까지 높은 단어에 대해 필터링
            # top-p는 일정 확률값 이상인 단어에 대해 필터링함
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        sent += gen.replace('▁', ' ')
    
    return sent
    

In [16]:
generate_sent('이때', gpt_model, greedy=True)

'이때문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에문에'

In [17]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.95)

'이때의까지의 리 일반적인 왕세 당시 얼굴이 쇄 비는  막 오 ‘ 쥐 파문에는 보내 막내 기상청은 발동 운동 한가지 또 주민 브로 굳이 서 공연뿐 이들이 ▲를부터는 절만 N 강 도움 진한 보 연 50 문 갑자기문에 한, 가자 지정된 회의를 차를 니 아이들은문에, 미소를 장세까지만,인 이른바는 소득이문에도 개각까지 그는 위기관리를 류현까지 강 문이 파리 농장 ""까지 만나 몇몇 이강는 GDP 동맹 신하 중요한 일 페 수상한 여성 생보는엔 링 육 받은도의'

In [19]:
DATA_IN_PATH = './data_in/KOR/'
TRAIN_DATA_FILE = 'finetune_data.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE, encoding='utf-8').readlines()]

In [20]:
input_data = []
output_data = []

for s in sents:
    tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

In [21]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [22]:
gpt_model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

In [23]:
history = gpt_model.fit(input_data, output_data, 
                    batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                    validation_split=0.1)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x110f45d90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x110f45d90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


16/16 [==============================] - ETA: 0s - loss: 3.1946 - accuracy_function: 0.0981

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


16/16 [==============================] - 109s 5s/step - loss: 3.1824 - accuracy_function: 0.0984 - val_loss: 2.4842 - val_accuracy_function: 0.1152
Epoch 2/10
16/16 [==============================] - 74s 5s/step - loss: 2.6140 - accuracy_function: 0.1221 - val_loss: 2.4035 - val_accuracy_function: 0.1335
Epoch 3/10
16/16 [==============================] - 75s 5s/step - loss: 2.3162 - accuracy_function: 0.1387 - val_loss: 2.3850 - val_accuracy_function: 0.1471
Epoch 4/10
16/16 [==============================] - 78s 5s/step - loss: 1.9711 - accuracy_function: 0.1502 - val_loss: 2.3892 - val_accuracy_function: 0.1597
Epoch 5/10
16/16 [==============================] - 77s 5s/step - loss: 1.8417 - accuracy_function: 0.1640 - val_loss: 2.4304 - val_accuracy_function: 0.1710
Epoch 6/10
16/16 [==============================] - 76s 5s/step - loss: 1.5958 - accuracy_function: 0.1748 - val_loss: 2.4907 - val_accuracy_function: 0.1827
Epoch 7/10
16/16 [==============================] - 78s 5s/ste

In [24]:
DATA_OUT_PATH = './data_out'
model_name = "tf2_gpt2_finetuned_model"

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./data_out/tf2_gpt2_finetuned_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [25]:
generate_sent('이때', gpt_model, greedy=True)

'이때에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에에'

In [26]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.95)

'이때에 그 뒤에서에 의 미군문에를만의문에 마침 김 의사는문에 문에를 문 문 취 곰에 신문에문에 주 병 행 벌는 술집에 김씨의 미스 문 레 맥주에에 눈물이에 손까지문에 옆에에 오정 헛, 교 은 마침 아 탁 문 그의의에문에에를문에 바일 마침에가에 다리에 김 김 문에 안에 유에 조 병 십자가 불에에 김 김에까지 문문에 김에 결 김에 누군가가에만가를'